In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [3]:
df_merged = pd.read_csv('3 VADERphase.csv')
df_merged


,Unnamed: 0.1,Unnamed: 0,Username,Text,Date Created,Number of Likes,Number of Retweet,Number of Replies,Source of Tweet,User ID,Conversation ID,status,Category,translated,translated cleaned,sentiment Scores,sentiment,sentiment_label
0,2,2,XavierNaxa,Melihat cara kerajaan mahupun pembangkang meny...,2021-12-26 07:16:38+00:00,3047,2239,50,Twitter for iPhone,1475002598327394306,1475002598327394306,Main post,Technology,Seeing how the government and the opposition s...,government opposition solved flood issue times...,"{'neg': 0.126, 'neu': 0.651, 'pos': 0.223, 'co...",0.2732,Positive
1,3,3,XavierNaxa,Penatlah dapat sorang menteri perasan cantik n...,2021-12-19 13:59:40+00:00,10386,6531,57,Twitter Web App,1472567309940129792,1472567309940129792,Main post,Technology,Tired of getting a minister to notice this bea...,tired minister notice beauty,"{'neg': 0.333, 'neu': 0.23, 'pos': 0.437, 'com...",0.2263,Positive
2,4,4,XavierNaxa,Apakah produk Apple yang berjaya anda miliki p...,2021-12-17 16:42:15+00:00,397,60,293,Twitter for iPhone,1471883452324597762,1471883452324597762,Main post,Technology,What are the successful Apple products you hav...,successful apple products year,"{'neg': 0.0, 'neu': 0.441, 'pos': 0.559, 'comp...",0.5859,Positive
3,5,5,XavierNaxa,Promosi penukaran bateri iPhone 8 dan iPhone 8...,2021-12-14 10:57:18+00:00,921,532,63,Twitter for iPhone,1470709477196193796,1470709477196193796,Main post,Technology,iPhone 8 and iPhone 8 Plus battery replacement...,iphone iphone battery replacement promotion lo...,"{'neg': 0.111, 'neu': 0.741, 'pos': 0.148, 'co...",0.1779,Positive
4,6,6,XavierNaxa,Apa barangan menarik sempena 12.12 yang telah ...,2021-12-11 16:58:35+00:00,225,28,67,Twitter for iPhone,1469713235125309445,1469713235125309445,Main post,Technology,What interesting items in conjunction with 12....,interesting items conjunction bought share,"{'neg': 0.0, 'neu': 0.38, 'pos': 0.62, 'compou...",0.5994,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76582,152259,152259,SuriaArul,@SyedSaddiq Ridiculous,2021-09-29 14:02:13+00:00,0,0,0,Twitter for iPhone,1443214534294966280,1442708708308373509,reply,Politics,@SyedSaddiq Ridiculous,ridiculous,"{'neg': 1.0, 'neu': 0.0, 'pos': 0.0, 'compound...",-0.3612,Negative
76583,152260,152260,rotiPATsegi,"@SyedSaddiq Guna untuk sekolah, hospital, ban...",2021-09-29 15:35:21+00:00,0,0,0,Twitter for Android,1443237972267061248,1442708708308373509,reply,Politics,"@SyedSaddiq Use it for schools, hospitals, hel...",schools hospitals people beneficial wasteful a...,"{'neg': 0.0, 'neu': 0.674, 'pos': 0.326, 'comp...",0.4404,Positive
76584,152261,152261,RinMaLiK1,@SyedSaddiq Kebanyakan menteri hanya nak penuh...,2021-09-29 16:36:23+00:00,0,0,0,Twitter for Android,1443253331653132288,1442708708308373509,reply,Politics,@SyedSaddiq Most ministers only want to meet m...,ministers meet work people excuse actions immo...,"{'neg': 0.242, 'neu': 0.484, 'pos': 0.274, 'co...",-0.1531,Negative
76585,152263,152263,Mrzach17,@SyedSaddiq @EZIARIFFIN Pergi laaaaa tlg rakya...,2021-09-29 23:42:04+00:00,0,0,0,Twitter for Android,1443360459726356482,1442708708308373509,reply,Politics,@SyedSaddiq @EZIARIFFIN Let's go to the people...,people people people sick businesses bankrupt ...,"{'neg': 0.574, 'neu': 0.426, 'pos': 0.0, 'comp...",-0.7964,Negative


In [4]:
df_entrepreneur = df_merged[df_merged["Category"] == "Entrepreneur"]
df_entertainer = df_merged[df_merged["Category"] == "Entertainer"]
df_technology = df_merged[df_merged["Category"] == "Technology"]
df_celebrity = df_merged[df_merged["Category"] == "Celebrity"]
df_politics = df_merged[df_merged["Category"] == "Politics"]
df_others = df_merged[df_merged["Category"] == "Fitness and others"]


# MERGED


In [5]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Replace NaN values with an empty string
df_merged['translated cleaned'] =df_merged['translated cleaned'].fillna('')

# Define the pipeline with CountVectorizer, SVM classifier, and parameter grid for hyperparameter tuning
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', SVC()),
])

parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'clf__kernel': ('linear', 'rbf'),
    'clf__C': [0.1, 1, 10,1000],
    'clf__gamma': [0.01,0.02,0.03,0.04,0.05,0.06,0.1, 1],
}

# Define the GridSearchCV object with 5-fold cross-validation
grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1)

# Split the data into training and testing sets equally for positive and negative sentiment
X_pos = df_merged[df_merged['sentiment'] > 0]['translated cleaned']
X_neg = df_merged[df_merged['sentiment'] <= 0]['translated cleaned']
y_pos = df_merged[df_merged['sentiment'] > 0]['sentiment'].apply(lambda score: 1)
y_neg = df_merged[df_merged['sentiment'] <= 0]['sentiment'].apply(lambda score: 0)

X_train_pos, X_test_pos, y_train_pos, y_test_pos = train_test_split(X_pos, y_pos, test_size=0.2, random_state=42, stratify=y_pos)
X_train_neg, X_test_neg, y_train_neg, y_test_neg = train_test_split(X_neg, y_neg, test_size=0.2, random_state=42, stratify=y_neg)

X_train_merge = pd.concat([X_train_pos, X_train_neg])
X_test_merge = pd.concat([X_test_pos, X_test_neg])
y_train_merge = pd.concat([y_train_pos, y_train_neg])
y_test_merge = pd.concat([y_test_pos, y_test_neg])

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train_merge, y_train_merge)

# Print the best hyperparameters and corresponding score
print(f"Best hyperparameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Fit a final model on the entire training set using the best hyperparameters
pipeline.set_params(**grid_search.best_params_)
pipeline.fit(X_train_merge, y_train_merge)

# Use the final model to make predictions on the test set
y_pred_merge = pipeline.predict(X_test_merge)

# Evaluate the performance of the final model on the test set
accuracy_merge = accuracy_score(y_test_merge, y_pred_merge)
precision_merge = precision_score(y_test_merge, y_pred_merge)
recall_merge = recall_score(y_test_merge, y_pred_merge)
f1_merge = f1_score(y_test_merge, y_pred_merge)

print(f"Accuracy: {accuracy_merge}")
print(f"Precision: {precision_merge}")
print(f"Recall: {recall_merge}")
print(f"F1 score: {f1_merge}")


Accuracy: 0.957370413892153
Precision: 0.9629402756508423
Recall: 0.970170746759926
F1 score: 0.9665419890352002


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Fit a final model on the entire training set using the best hyperparameters
pipeline.set_params(**grid_search.best_params_)
pipeline.fit(X_train_merge, y_train_merge)

# Use the final model to make predictions on the test set
y_pred_merge = pipeline.predict(X_test_merge)

# Evaluate the performance of the final model on the test set
accuracy_merge = accuracy_score(y_test_merge, y_pred_merge)
precision_merge = precision_score(y_test_merge, y_pred_merge, average=None)
recall_merge = recall_score(y_test_merge, y_pred_merge, average=None)
f1_merge = f1_score(y_test_merge, y_pred_merge, average=None)

# Create a dictionary to store the metric values
metrics_dict = {"Label": ["Positive", "Negative"],
                "Precision": precision_merge,
                "Recall": recall_merge,
                "F1 Score": f1_merge}

# Convert the dictionary into a dataframe
metrics_df = pd.DataFrame.from_dict(metrics_dict)

# Set the "Label" column as the index
metrics_df.set_index("Label", inplace=True)

# Print the dataframe
print(metrics_df)


          Precision    Recall  F1 Score
Label                                  
Positive   0.946029  0.930307  0.938103
Negative   0.960265  0.969451  0.964836


In [ ]:
metrics_df.to_csv("merged_metrics.csv", index=False)

In [ ]:
output_merge = pd.DataFrame({'text': X_test_merge, 'vader_label': y_test_merge, 'predicted_label': y_pred_merge})
output_merge.to_csv("svm_merge.csv")

In [ ]:
# Create a new column in the test data portion of df_merged and assign the predicted labels to it
df_merged.loc[y_test_merge.index, 'SVM Predicted Label'] = y_pred_merge

# Save the updated DataFrame to a CSV file
df_merged.to_csv('updated_df_merged.csv', index=False)


# TECHNOLOGY

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle

# Replace NaN values with an empty string
df_technology['translated cleaned'] = df_technology['translated cleaned'].fillna('')

# Split the data into positive and negative subsets
positive_subset = df_technology[df_technology['sentiment'] > 0]
negative_subset = df_technology[df_technology['sentiment'] <= 0]

# Shuffle the data to ensure a random split
positive_subset = shuffle(positive_subset, random_state=42)
negative_subset = shuffle(negative_subset, random_state=42)

# Determine the smaller subset size
subset_size = min(len(positive_subset), len(negative_subset))

# Split the subsets into training and testing sets with equal positive and negative class sizes
train_size = int(0.8 * subset_size)
test_size = subset_size - train_size

X_train_positive = positive_subset['translated cleaned'][:train_size]
X_test_positive = positive_subset['translated cleaned'][train_size:train_size + test_size]
y_train_positive = positive_subset['sentiment'][:train_size].apply(lambda score: 1 if score > 0 else 0)
y_test_positive = positive_subset['sentiment'][train_size:train_size + test_size].apply(lambda score: 1 if score > 0 else 0)

X_train_negative = negative_subset['translated cleaned'][:train_size]
X_test_negative = negative_subset['translated cleaned'][train_size:train_size + test_size]
y_train_negative = negative_subset['sentiment'][:train_size].apply(lambda score: 1 if score > 0 else 0)
y_test_negative = negative_subset['sentiment'][train_size:train_size + test_size].apply(lambda score: 1 if score > 0 else 0)

X_train_tech = pd.concat([X_train_positive, X_train_negative])
X_test_tech = pd.concat([X_test_positive, X_test_negative])
y_train_tech = pd.concat([y_train_positive, y_train_negative])
y_test_tech = pd.concat([y_test_positive, y_test_negative])

# Define the pipeline with CountVectorizer, SVM classifier, and parameter grid for hyperparameter tuning
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', SVC()),
])

parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'clf__kernel': ('linear', 'rbf'),
    'clf__C': [0.1, 1, 10, 1000],
    'clf__gamma': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.1, 1],
}

# Define the GridSearchCV object with 5-fold cross-validation
grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train_tech, y_train_tech)

# Print the best hyperparameters and corresponding score
print(f"Best hyperparameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")


C:\Users\Fikri\AppData\Local\Temp\ipykernel_5052\3846090957.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_technology['translated cleaned'] = df_technology['translated cleaned'].fillna('')
C:\Users\Fikri\AppData\Local\Temp\ipykernel_5052\3846090957.py:23: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  X_train_positive = positive_subset['translated cleaned'][:train_size]
C:\Users\Fikri\AppData\Local\Temp\ipykernel_5052\3846090957.py:24: FutureWarning: The behavior of `series[i:j]` with a

Best hyperparameters: {'clf__C': 1000, 'clf__gamma': 0.01, 'clf__kernel': 'rbf', 'vect__ngram_range': (1, 1)}
Best score: 0.8402032984130277


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Fit a final model on the entire training set using the best hyperparameters
pipeline.set_params(**grid_search.best_params_)
pipeline.fit(X_train_tech, y_train_tech)

# Use the final model to make predictions on the test set
y_pred_tech = pipeline.predict(X_test_tech)

# Evaluate the performance of the final model on the test set
accuracy_tech = accuracy_score(y_test_tech, y_pred_tech)
precision_tech = precision_score(y_test_tech, y_pred_tech)
recall_tech = recall_score(y_test_tech, y_pred_tech)
f1_tech = f1_score(y_test_tech, y_pred_tech)

print(f"Accuracy: {accuracy_tech}")
print(f"Precision: {precision_tech}")
print(f"Recall: {recall_tech}")
print(f"F1 score: {f1_tech}")


Accuracy: 0.8205128205128205
Precision: 0.8140703517587939
Recall: 0.8307692307692308
F1 score: 0.8223350253807108


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Fit a final model on the entire training set using the best hyperparameters
pipeline.set_params(**grid_search.best_params_)
pipeline.fit(X_train_tech, y_train_tech)

# Use the final model to make predictions on the test set
y_pred_tech = pipeline.predict(X_test_tech)

# Evaluate the performance of the final model on the test set
accuracy_tech = accuracy_score(y_test_tech, y_pred_tech)
precision_tech = precision_score(y_test_tech, y_pred_tech, average=None)
recall_tech = recall_score(y_test_tech, y_pred_tech, average=None)
f1_tech = f1_score(y_test_tech, y_pred_tech, average=None)

# Create a dictionary to store the metric values
metrics_dict = {"Label": ["Positive", "Negative"],
                "Precision": precision_tech,
                "Recall": recall_tech,
                "F1 Score": f1_tech}

# Convert the dictionary into a dataframe
metrics_df = pd.DataFrame.from_dict(metrics_dict)

# Set the "Label" column as the index
metrics_df.set_index("Label", inplace=True)

# Print the dataframe
print(metrics_df)

          Precision    Recall  F1 Score
Label                                  
Positive   0.830000  0.851282  0.840506
Negative   0.847368  0.825641  0.836364


In [ ]:
metrics_df.to_csv("tech_metrics.csv", index=False)

In [ ]:
output_tech = pd.DataFrame({'text': X_test_tech, 'vader_label': y_test_tech, 'predicted_label': y_pred_tech})
output_tech.to_csv("svm_tech.csv")

In [ ]:
# Create a new column in the test data portion of df_merged and assign the predicted labels to it
df_technology.loc[y_test_tech.index, 'SVM Predicted Label'] = y_pred_tech

# Save the updated DataFrame to a CSV file
df_technology.to_csv('updated_df_tech.csv', index=False)


C:\Users\Fikri\AppData\Local\Temp\ipykernel_5052\2872777365.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_technology.loc[y_test_tech.index, 'SVM Predicted Label'] = y_pred_tech


# POLITICS


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle

# Replace NaN values with an empty string
df_politics['translated cleaned'] = df_politics['translated cleaned'].fillna('')

# Split the data into positive and negative sentiment
pos_data = df_politics[df_politics['sentiment'] > 0]
neg_data = df_politics[df_politics['sentiment'] <= 0]

# Shuffle the data and split into training and testing sets with equal amounts of positive and negative sentiment
pos_train, pos_test = train_test_split(pos_data, test_size=0.2, random_state=42)
neg_train, neg_test = train_test_split(neg_data, test_size=0.2, random_state=42)
train = shuffle(pd.concat([pos_train, neg_train]))
test = shuffle(pd.concat([pos_test, neg_test]))

# Split the training and testing data into X and y
X_train_pol = train['translated cleaned']
y_train_pol = train['sentiment'].apply(lambda score: 1 if score > 0 else 0)
X_test_pol = test['translated cleaned']
y_test_pol = test['sentiment'].apply(lambda score: 1 if score > 0 else 0)

# Define the pipeline with CountVectorizer, SVM classifier, and parameter grid for hyperparameter tuning
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', SVC()),
])

parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'clf__kernel': ('linear', 'rbf'),
    'clf__C': [0.1, 1, 10, 1000],
    'clf__gamma': [0.01, 0.1, 1, 10, 100],
}

# Define the GridSearchCV object with 5-fold cross-validation
grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train_pol, y_train_pol)

# Print the best hyperparameters and corresponding score
print(f"Best hyperparameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")


C:\Users\Fikri\AppData\Local\Temp\ipykernel_5052\2725342792.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_politics['translated cleaned'] = df_politics['translated cleaned'].fillna('')


Best hyperparameters: {'clf__C': 10, 'clf__gamma': 0.01, 'clf__kernel': 'rbf', 'vect__ngram_range': (1, 1)}
Best score: 0.9316204554076689


In [ ]:
# Fit a final model on the entire training set using the best hyperparameters
pipeline.set_params(**grid_search.best_params_)
pipeline.fit(X_train_pol, y_train_pol)

# Use the final model to make predictions on the test set
y_pred_pol = pipeline.predict(X_test_pol)

# Evaluate the performance of the final model on the test set
accuracy_pol = accuracy_score(y_test_pol, y_pred_pol)
precision_pol = precision_score(y_test_pol, y_pred_pol)
recall_pol = recall_score(y_test_pol, y_pred_pol)
f1_pol = f1_score(y_test_pol, y_pred_pol)

print(f"Accuracy: {accuracy_pol}")
print(f"Precision: {precision_pol}")
print(f"Recall: {recall_pol}")
print(f"F1 score: {f1_pol}")


Accuracy: 0.9353798710400897
Precision: 0.9389349112426035
Recall: 0.9513189448441247
F1 score: 0.9450863609291246


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Fit a final model on the entire training set using the best hyperparameters
pipeline.set_params(**grid_search.best_params_)
pipeline.fit(X_train_pol, y_train_pol)

# Use the final model to make predictions on the test set
y_pred_pol = pipeline.predict(X_test_pol)

# Evaluate the performance of the final model on the test set
accuracy_pol = accuracy_score(y_test_pol, y_pred_pol)
precision_pol = precision_score(y_test_pol, y_pred_pol, average=None)
recall_pol = recall_score(y_test_pol, y_pred_pol, average=None)
f1_pol = f1_score(y_test_pol, y_pred_pol, average=None)

# Create a dictionary to store the metric values
metrics_dict = {"Label": ["Positive", "Negative"],
                "Precision": precision_pol,
                "Recall": recall_pol,
                "F1 Score": f1_pol}

# Convert the dictionary into a dataframe
metrics_df = pd.DataFrame.from_dict(metrics_dict)

# Set the "Label" column as the index
metrics_df.set_index("Label", inplace=True)

# Print the dataframe
print(metrics_df)

          Precision    Recall  F1 Score
Label                                  
Positive   0.930217  0.912955  0.921505
Negative   0.938935  0.951319  0.945086


In [ ]:
metrics_df.to_csv("pol_metrics.csv", index=False)

In [ ]:
output_pol = pd.DataFrame({'text': X_test_pol, 'vader_label': y_test_pol, 'predicted_label': y_pred_pol})
output_pol.to_csv("svm_politic.csv")

In [ ]:
# Create a new column in the test data portion of df_merged and assign the predicted labels to it
df_politics.loc[y_test_pol.index, 'SVM Predicted Label'] = y_pred_pol

# Save the updated DataFrame to a CSV file
df_politics.to_csv('updated_df_pol.csv', index=False)


C:\Users\Fikri\AppData\Local\Temp\ipykernel_5052\496997231.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_politics.loc[y_test_pol.index, 'SVM Predicted Label'] = y_pred_pol


# ENTREPRENEUR

In [ ]:


from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle

# Replace NaN values with an empty string
df_entrepreneur['translated cleaned'] = df_entrepreneur['translated cleaned'].fillna('')

# Split the data into positive and negative sentiment
pos_data = df_entrepreneur[df_entrepreneur['sentiment'] > 0]
neg_data = df_entrepreneur[df_entrepreneur['sentiment'] <= 0]

# Shuffle the data and split into training and testing sets with equal amounts of positive and negative sentiment
pos_train, pos_test = train_test_split(pos_data, test_size=0.2, random_state=42)
neg_train, neg_test = train_test_split(neg_data, test_size=0.2, random_state=42)
train = shuffle(pd.concat([pos_train, neg_train]))
test = shuffle(pd.concat([pos_test, neg_test]))

# Split the training and testing data into X and y
X_train_entrep = train['translated cleaned']
y_train_entrep = train['sentiment'].apply(lambda score: 1 if score > 0 else 0)
X_test_entrep = test['translated cleaned']
y_test_entrep = test['sentiment'].apply(lambda score: 1 if score > 0 else 0)

# Define the pipeline with CountVectorizer, SVM classifier, and parameter grid for hyperparameter tuning
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', SVC()),
])

parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'clf__kernel': ('linear', 'rbf'),
    'clf__C': [0.1, 1, 10, 1000],
    'clf__gamma': [0.01, 0.1, 1, 10, 100],
}

# Define the GridSearchCV object with 5-fold cross-validation
grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train_entrep, y_train_entrep)

# Print the best hyperparameters and corresponding score
print(f"Best hyperparameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")



C:\Users\Fikri\AppData\Local\Temp\ipykernel_5052\1217829535.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_entrepreneur['translated cleaned'] = df_entrepreneur['translated cleaned'].fillna('')


Best hyperparameters: {'clf__C': 1, 'clf__gamma': 0.01, 'clf__kernel': 'linear', 'vect__ngram_range': (1, 1)}
Best score: 0.9324212996266228


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Fit a final model on the entire training set using the best hyperparameters
pipeline.set_params(**grid_search.best_params_)
pipeline.fit(X_train_entrep, y_train_entrep)

# Use the final model to make predictions on the test set
y_pred_entrep = pipeline.predict(X_test_entrep)

# Evaluate the performance of the final model on the test set
accuracy_entrep = accuracy_score(y_test_entrep, y_pred_entrep)
precision_entrep = precision_score(y_test_entrep, y_pred_entrep)
recall_entrep = recall_score(y_test_entrep, y_pred_entrep)
f1_entrep = f1_score(y_test_entrep, y_pred_entrep)

print(f"Accuracy: {accuracy_entrep}")
print(f"Precision: {precision_entrep}")
print(f"Recall: {recall_entrep}")
print(f"F1 score: {f1_entrep}")

Accuracy: 0.9341438703140831
Precision: 0.9492753623188406
Recall: 0.9619091326296466
F1 score: 0.9555504900843401


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Fit a final model on the entire training set using the best hyperparameters
pipeline.set_params(**grid_search.best_params_)
pipeline.fit(X_train_entrep, y_train_entrep)

# Use the final model to make predictions on the test set
y_pred_entrep = pipeline.predict(X_test_entrep)

# Evaluate the performance of the final model on the test set
accuracy_entrep = accuracy_score(y_test_entrep, y_pred_entrep)
precision_entrep = precision_score(y_test_entrep, y_pred_entrep, average=None)
recall_entrep = recall_score(y_test_entrep, y_pred_entrep, average=None)
f1_entrep= f1_score(y_test_entrep, y_pred_entrep, average=None)

# Create a dictionary to store the metric values
metrics_dict = {"Label": ["Positive", "Negative"],
                "Precision": precision_entrep,
                "Recall": recall_entrep,
                "F1 Score": f1_entrep}

# Convert the dictionary into a dataframe
metrics_df = pd.DataFrame.from_dict(metrics_dict)

# Set the "Label" column as the index
metrics_df.set_index("Label", inplace=True)

# Print the dataframe
print(metrics_df)

          Precision    Recall  F1 Score
Label                                  
Positive   0.903794  0.852941  0.877632
Negative   0.948268  0.967416  0.957746


In [ ]:
metrics_df.to_csv("entrep_metrics.csv", index=False)

In [ ]:
output_entrep = pd.DataFrame({'text': X_test_entrep, 'vader_label': y_test_entrep, 'predicted_label': y_pred_entrep})
output_entrep.to_csv("svm_entrep.csv")

In [ ]:
# Create a new column in the test data portion of df_merged and assign the predicted labels to it
df_entrepreneur.loc[y_test_entrep.index, 'SVM Predicted Label'] = y_pred_entrep

# Save the updated DataFrame to a CSV file
df_entrepreneur.to_csv('updated_df_entrep.csv', index=False)

C:\Users\Fikri\AppData\Local\Temp\ipykernel_5052\2966863844.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_entrepreneur.loc[y_test_entrep.index, 'SVM Predicted Label'] = y_pred_entrep


# CELEBRITY 

In [7]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
from imblearn.over_sampling import SMOTE

# Replace NaN values with an empty string
df_celebrity['translated cleaned'] = df_celebrity['translated cleaned'].fillna('')

model = SVC()

# Define the hyperparameters to search over
param_grid = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
              'C': [0.1, 1, 10, 100],
              'gamma': [0.01, 0.1, 1, 10],
              'probability': [True]}



# Define the vectorizer
vectorizer = TfidfVectorizer(use_idf=True, max_features=10000)

# Split the data into positive and negative sentiment
pos_data = df_celebrity[df_celebrity['sentiment'] > 0]
neg_data = df_celebrity[df_celebrity['sentiment'] <= 0]

# Shuffle the data and split into training and testing sets with
# equal amounts of positive and negative sentiment
pos_train, pos_test = train_test_split(pos_data, test_size=0.2, random_state=42)
neg_train, neg_test = train_test_split(neg_data, test_size=0.2, random_state=42)
train = shuffle(pd.concat([pos_train, neg_train]))
test = shuffle(pd.concat([pos_test, neg_test]))

# Split the training and testing data into X and y
X_train_celeb = train['translated cleaned']
y_train_celeb = train['sentiment'].apply(lambda score: 1 if score > 0 else 0)
X_test_celeb = test['translated cleaned']
y_test_celeb = test['sentiment'].apply(lambda score: 1 if score > 0 else 0)

# Fit and transform the training data
X_train_celeb = vectorizer.fit_transform(X_train_celeb)

# Perform oversampling on the training data using SMOTE
sm = SMOTE(random_state=42)
X_train_celeb_smote, y_train_celeb_smote = sm.fit_resample(X_train_celeb, y_train_celeb)

# Define the GridSearchCV object with 5-fold cross-validation
grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train_celeb, y_train_celeb)

# Print the best hyperparameters and corresponding score
print(f"Best hyperparameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")


C:\Users\Fikri\AppData\Local\Temp\ipykernel_20240\4010487705.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_celebrity['translated cleaned'] = df_celebrity['translated cleaned'].fillna('')


Best hyperparameters: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf', 'probability': True}
Best score: 0.8677201531500174


In [8]:
value_counts = df_celebrity['sentiment_label'].value_counts()
value_counts


Positive    818
Negative    242
Name: sentiment_label, dtype: int64

In [9]:
from sklearn.metrics import confusion_matrix
# Transform the test data using the same vectorizer used for the training data
X_test_celeb = vectorizer.transform(X_test_celeb)

# Make predictions using the best model obtained from the grid search
best_model = grid_search.best_estimator_
y_pred_celeb = best_model.predict(X_test_celeb)

# Compute the precision, recall, f1-score, and support for positive and negative sentiment
from sklearn.metrics import classification_report

report = classification_report(y_test_celeb, y_pred_celeb)
print(report)

# Compute the confusion matrix
cm = confusion_matrix(y_test_celeb, y_pred_celeb)

# Print the classification report with the confusion matrix
report = classification_report(y_test_celeb, y_pred_celeb)

print("Confusion matrix:")
print("              Predicted")
print("             |  0  |  1  |")
print(f"True    0   | {cm[0][0]}  |  {cm[0][1]}  |")
print(f"        1   | {cm[1][0]}  |  {cm[1][1]}  |")

              precision    recall  f1-score   support

           0       0.72      0.59      0.65        49
           1       0.88      0.93      0.91       164

    accuracy                           0.85       213
   macro avg       0.80      0.76      0.78       213
weighted avg       0.85      0.85      0.85       213

Confusion matrix:
              Predicted
             |  0  |  1  |
True    0   | 29  |  20  |
        1   | 11  |  153  |


In [10]:
report

'              precision    recall  f1-score   support\n\n           0       0.72      0.59      0.65        49\n           1       0.88      0.93      0.91       164\n\n    accuracy                           0.85       213\n   macro avg       0.80      0.76      0.78       213\nweighted avg       0.85      0.85      0.85       213\n'

In [10]:
metrics_df.to_csv("celeb_metrics.csv", index=False)

NameError: name 'metrics_df' is not defined

In [ ]:
output_celeb = pd.DataFrame({'text': X_test_celeb, 'vader_label': y_test_celeb, 'predicted_label': y_pred_celeb})
output_celeb.to_csv("svm_celeb.csv")

In [ ]:
# Create a new column in the test data portion of df_merged and assign the predicted labels to it
df_celebrity.loc[y_test_celeb.index, 'SVM Predicted Label'] = y_pred_celeb

# Save the updated DataFrame to a CSV file
df_celebrity.to_csv('updated_df_celeb.csv', index=False)

# ENTERTAINER

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle

# Replace NaN values with an empty string
df_entertainer['translated cleaned'] =df_entertainer['translated cleaned'].fillna('')

# Define the pipeline with CountVectorizer, SVM classifier, and parameter grid for hyperparameter tuning
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', SVC()),
])

parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'clf__kernel': ('linear', 'rbf'),
    'clf__C': [0.1, 1, 10, 1000],
    'clf__gamma': [0.01, 0.1, 1, 10, 100],
}

# Define the GridSearchCV object with 5-fold cross-validation
grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1)

# Split the data into positive and negative sentiment
pos_data = df_entertainer[df_entertainer['sentiment'] > 0]
neg_data = df_entertainer[df_entertainer['sentiment'] <= 0]

# Shuffle the data and split into training and testing sets with equal amounts of positive and negative sentiment
pos_train, pos_test = train_test_split(pos_data, test_size=0.2, random_state=42)
neg_train, neg_test = train_test_split(neg_data, test_size=0.2, random_state=42)
train = shuffle(pd.concat([pos_train, neg_train]))
test = shuffle(pd.concat([pos_test, neg_test]))

# Split the training and testing data into X and y
X_train_enter = train['translated cleaned']
y_train_enter = train['sentiment'].apply(lambda score: 1 if score > 0 else 0)
X_test_enter= test['translated cleaned']
y_test_enter= test['sentiment'].apply(lambda score: 1 if score > 0 else 0)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train_enter, y_train_enter)

# Print the best hyperparameters and corresponding score
print(f"Best hyperparameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")


C:\Users\Fikri\AppData\Local\Temp\ipykernel_5052\2395349068.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_entertainer['translated cleaned'] =df_entertainer['translated cleaned'].fillna('')


Best hyperparameters: {'clf__C': 1, 'clf__gamma': 0.01, 'clf__kernel': 'linear', 'vect__ngram_range': (1, 2)}
Best score: 0.9139531317772718


In [ ]:
from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle

# Replace NaN values with an empty string
df_entertainer['translated cleaned'] = df_entertainer['translated cleaned'].fillna('')

# Define the pipeline with CountVectorizer, SVM classifier, and parameter grid for hyperparameter tuning
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('smote', SMOTE()),
    ('clf', SVC()),
])

parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'clf__kernel': ('linear', 'rbf'),
    'clf__C': [0.1, 1, 10, 1000],
    'clf__gamma': [0.01, 0.1, 1, 10, 100],
}

# Define the GridSearchCV object with 5-fold cross-validation
grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1)

# Split the data into positive and negative sentiment
pos_data = df_entertainer[df_entertainer['sentiment'] > 0]
neg_data = df_entertainer[df_entertainer['sentiment'] <= 0]

# Shuffle the data and split into training and testing sets with equal amounts of positive and negative sentiment
pos_train, pos_test = train_test_split(pos_data, test_size=0.2, random_state=42)
neg_train, neg_test = train_test_split(neg_data, test_size=0.2, random_state=42)
train = shuffle(pd.concat([pos_train, neg_train]))
test = shuffle(pd.concat([pos_test, neg_test]))

# Split the training and testing data into X and y
X_train_enter = train['translated cleaned']
y_train_enter = train['sentiment'].apply(lambda score: 1 if score > 0 else 0)
X_test_enter = test['translated cleaned']
y_test_enter = test['sentiment'].apply(lambda score: 1 if score > 0 else 0)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train_enter, y_train_enter)

# Print the best hyperparameters and corresponding score
print(f"Best hyperparameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")


C:\Users\Fikri\AppData\Local\Temp\ipykernel_5052\765056327.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_entertainer['translated cleaned'] = df_entertainer['translated cleaned'].fillna('')


Best hyperparameters: {'clf__C': 1, 'clf__gamma': 100, 'clf__kernel': 'linear', 'vect__ngram_range': (1, 1)}
Best score: 0.8949494923823466


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Fit a final model on the entire training set using the best hyperparameters
pipeline.set_params(**grid_search.best_params_)
pipeline.fit(X_train_enter, y_train_enter)

# Use the final model to make predictions on the test set
y_pred_enter = pipeline.predict(X_test_enter)

# Evaluate the performance of the final model on the test set
accuracy_enter = accuracy_score(y_test_enter, y_pred_enter)
precision_enter = precision_score(y_test_enter, y_pred_enter)
recall_enter = recall_score(y_test_enter, y_pred_enter)
f1_enter = f1_score(y_test_enter, y_pred_enter)

print(f"Accuracy: {accuracy_enter}")
print(f"Precision: {precision_enter}")
print(f"Recall: {recall_enter}")
print(f"F1 score: {f1_enter}")

Accuracy: 0.927159209157128
Precision: 0.9412244897959183
Recall: 0.9443079443079443
F1 score: 0.9427636958299264


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Fit a final model on the entire training set using the best hyperparameters
pipeline.set_params(**grid_search.best_params_)
pipeline.fit(X_train_enter, y_train_enter)

# Use the final model to make predictions on the test set
y_pred_enter= pipeline.predict(X_test_enter)

# Evaluate the performance of the final model on the test set
accuracy_enter = accuracy_score(y_test_enter, y_pred_enter)
precision_enter= precision_score(y_test_enter, y_pred_enter, average=None)
recall_enter = recall_score(y_test_enter, y_pred_enter, average=None)
f1_enter= f1_score(y_test_enter, y_pred_enter, average=None)

# Create a dictionary to store the metric values
metrics_dict = {"Label": ["Positive", "Negative"],
                "Precision": precision_enter,
                "Recall": recall_enter,
                "F1 Score": f1_enter}

# Convert the dictionary into a dataframe
metrics_df = pd.DataFrame.from_dict(metrics_dict)

# Set the "Label" column as the index
metrics_df.set_index("Label", inplace=True)

# Print the dataframe
print(metrics_df)

          Precision    Recall  F1 Score
Label                                  
Positive   0.888889  0.890157  0.889522
Negative   0.936885  0.936118  0.936501


In [ ]:
metrics_df.to_csv("enter_metrics.csv", index=False)

In [ ]:
output_enter = pd.DataFrame({'text': X_test_enter, 'vader_label': y_test_enter, 'predicted_label': y_pred_enter})
output_enter.to_csv("svm_enter.csv")

In [ ]:
# Create a new column in the test data portion of df_merged and assign the predicted labels to it
df_entertainer.loc[y_test_enter.index, 'SVM Predicted Label'] = y_pred_enter

# Save the updated DataFrame to a CSV file
df_entertainer.to_csv('updated_df_enter.csv', index=False)

C:\Users\Fikri\AppData\Local\Temp\ipykernel_5052\659843509.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_entertainer.loc[y_test_enter.index, 'SVM Predicted Label'] = y_pred_enter


# OTHERS


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle

# Replace NaN values with an empty string
df_others['translated cleaned'] =df_others['translated cleaned'].fillna('')

# Define the pipeline with CountVectorizer, SVM classifier, and parameter grid for hyperparameter tuning
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', SVC()),
])

parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'clf__kernel': ('linear', 'rbf'),
    'clf__C': [0.1, 1, 10, 1000],
    'clf__gamma': [0.01, 0.1, 1, 10, 100],
}

# Define the GridSearchCV object with 5-fold cross-validation
grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1)

# Split the data into positive and negative sentiment
pos_data = df_others[df_others['sentiment'] > 0]
neg_data = df_others[df_others['sentiment'] <= 0]

# Shuffle the data and split into training and testing sets with equal amounts of positive and negative sentiment
pos_train, pos_test = train_test_split(pos_data, test_size=0.2, random_state=42)
neg_train, neg_test = train_test_split(neg_data, test_size=0.2, random_state=42)
train = shuffle(pd.concat([pos_train, neg_train]))
test = shuffle(pd.concat([pos_test, neg_test]))

# Split the training and testing data into X and y
X_train_others= train['translated cleaned']
y_train_others= train['sentiment'].apply(lambda score: 1 if score > 0 else 0)
X_test_others= test['translated cleaned']
y_test_others= test['sentiment'].apply(lambda score: 1 if score > 0 else 0)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train_others, y_train_others)

# Print the best hyperparameters and corresponding score
print(f"Best hyperparameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")


C:\Users\Fikri\AppData\Local\Temp\ipykernel_5052\721486208.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_others['translated cleaned'] =df_others['translated cleaned'].fillna('')


Best hyperparameters: {'clf__C': 10, 'clf__gamma': 0.01, 'clf__kernel': 'rbf', 'vect__ngram_range': (1, 1)}
Best score: 0.8922958186591096


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Fit a final model on the entire training set using the best hyperparameters
pipeline.set_params(**grid_search.best_params_)
pipeline.fit(X_train_others, y_train_others)

# Use the final model to make predictions on the test set
y_pred_others = pipeline.predict(X_test_others)

# Evaluate the performance of the final model on the test set
accuracy_others = accuracy_score(y_test_others, y_pred_others)
precision_others = precision_score(y_test_others, y_pred_others)
recall_others = recall_score(y_test_others, y_pred_others)
f1_others = f1_score(y_test_others, y_pred_others)

print(f"Accuracy: {accuracy_others}")
print(f"Precision: {precision_others}")
print(f"Recall: {recall_others}")
print(f"F1 score: {f1_others}")

Accuracy: 0.9094503756425465
Precision: 0.922890103217972
Recall: 0.93711467324291
F1 score: 0.9299479963291526


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Fit a final model on the entire training set using the best hyperparameters
pipeline.set_params(**grid_search.best_params_)
pipeline.fit(X_train_others, y_train_others)

# Use the final model to make predictions on the test set
y_pred_others= pipeline.predict(X_test_others)

# Evaluate the performance of the final model on the test set
accuracy_others = accuracy_score(y_test_others, y_pred_others)
precision_others= precision_score(y_test_others, y_pred_others, average=None)
recall_others= recall_score(y_test_others, y_pred_others, average=None)
f1_others= f1_score(y_test_others, y_pred_others, average=None)

# Create a dictionary to store the metric values
metrics_dict = {"Label": ["Positive", "Negative"],
                "Precision": precision_others,
                "Recall": recall_others,
                "F1 Score": f1_others}

# Convert the dictionary into a dataframe
metrics_df = pd.DataFrame.from_dict(metrics_dict)

# Set the "Label" column as the index
metrics_df.set_index("Label", inplace=True)

# Print the dataframe
print(metrics_df)

          Precision    Recall  F1 Score
Label                                  
Positive   0.884354  0.859978  0.871996
Negative   0.922890  0.937115  0.929948


In [ ]:
metrics_df.to_csv("others_metrics.csv", index=False)

In [ ]:
output_others = pd.DataFrame({'text': X_test_others, 'vader_label': y_test_others, 'predicted_label': y_pred_others})
output_others.to_csv("svm_others.csv")

In [ ]:
# Create a new column in the test data portion of df_merged and assign the predicted labels to it
df_others.loc[y_test_others.index, 'SVM Predicted Label'] = y_pred_others

# Save the updated DataFrame to a CSV file
df_others.to_csv('updated_df_others.csv', index=False)

C:\Users\Fikri\AppData\Local\Temp\ipykernel_5052\3208066785.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_others.loc[y_test_others.index, 'SVM Predicted Label'] = y_pred_others
